In [37]:
import tensorflow as tf

In [38]:
# 1. load vdj dataset
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import joblib
vdjdb = pd.read_csv("../processed_data/vdjdb_preprocessd_nonull.txt", sep = "\t")
vdjdb.head()

,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,antigen.gene,antigen.species,vdjdb.score
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,2
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,2
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,2
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,2
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,2


In [39]:
data=vdjdb[['gene','cdr3','v.segm','j.segm','mhc.a','mhc.b']]
target=vdjdb['antigen.epitope']
# 3. feature engineering

# 3.1 feature extraction

# 1) Dictionary feature extraction
from sklearn.feature_extraction import DictVectorizer
data=data.to_dict(orient="records") 
DicVec_transfer = DictVectorizer(sparse=False)
data = DicVec_transfer.fit_transform(data)
print("Dictionary feature extraction return:\n", data)
print("DictVectorized feature names：\n", DicVec_transfer.get_feature_names_out())

Dictionary feature extraction return:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
DictVectorized feature names：
 ['cdr3=CAAAASGGSYIPTF' 'cdr3=CAAADEEIGNQPQHF' 'cdr3=CAAALNTNAGKSTF' ...
 'v.segm=TRBV7-9*03' 'v.segm=TRBV9*01' 'v.segm=TRBV9*02']


In [40]:
n_c=np.unique(target).shape[0]
print(n_c)

452


In [41]:
target2=vdjdb[['antigen.epitope']].to_dict(orient="records") 
DicVec_transfer2 = DictVectorizer(sparse=False)
target2 = DicVec_transfer2.fit_transform(target2)
print(target2)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
from sklearn import cluster
target2_Kmeans= cluster.KMeans(n_clusters = n_c, random_state = 42)
target2_cluster_labels = target2_Kmeans.fit_predict(target2)
print(target2_cluster_labels)

E:\Programs\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[16 16 16 ... 21 21 21]


In [43]:
print(np.unique(target2_cluster_labels).shape)

(452,)


In [44]:
# 2. split dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, target2_cluster_labels,
random_state=22)
print("x_train:\n", x_train.shape)
print("x_test:\n", x_test.shape)
print("y_train:\n", y_train.shape)
print("y_test:\n", y_test.shape)

x_train:
 (8498, 7445)
x_test:
 (2833, 7445)
y_train:
 (8498,)
y_test:
 (2833,)


In [45]:
# 转换x的数据类型，否则后面矩阵相乘时会因数据类型不一致报错
x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [46]:
#y_train = tf.cast(x_train, tf.float32)
#y_test = tf.cast(x_test, tf.float32)
#print(y_train.shape)
#target2=target.to_dict(orient="records") 
#DicVec_transfer2 = DictVectorizer(sparse=False)
#target2 = DicVec_transfer2.fit_transform(target)
print(y_train)

[62  0  6 ... 19  8 24]


In [47]:
# from_tensor_slices函数使输入特征和标签值一一对应。（把数据集分批次，每个批次batch组数据）
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [48]:
# 生成神经网络的参数，输入层为7445个输入节点；故输出层为神经元个数为一共有多少种epitope
# 用tf.Variable()标记参数可训练
# 使用seed使每次生成的随机数相同（方便教学，使大家结果都一致，在现实使用时不写seed）
w1 = tf.Variable(tf.random.truncated_normal([7445, n_c], stddev=0.1, seed=1))
b1 = tf.Variable(tf.random.truncated_normal([n_c], stddev=0.1, seed=1))

In [49]:
lr = 0.1  # 学习率为0.1
train_loss_results = []  # 将每轮的loss记录在此列表中，为后续画loss曲线提供数据
test_acc = []  # 将每轮的acc记录在此列表中，为后续画acc曲线提供数据
epoch = 500  # 循环500轮
loss_all = 0  # 每轮分4个step，loss_all记录四个step生成的4个loss的和

In [50]:
# 训练部分
for epoch in range(epoch):  #数据集级别的循环，每个epoch循环一次数据集
    for step, (x_train, y_train) in enumerate(train_db):  #batch级别的循环 ，每个step循环一个batch
        with tf.GradientTape() as tape:  # with结构记录梯度信息
            y = tf.matmul(x_train, w1) + b1  # 神经网络乘加运算
            y = tf.nn.softmax(y)  # 使输出y符合概率分布（此操作后与独热码同量级，可相减求loss）
            y_ = tf.one_hot(y_train, depth=3)  # 将标签值转换为独热码格式，方便计算loss和accuracy
            loss = tf.reduce_mean(tf.square(y_ - y))  # 采用均方误差损失函数mse = mean(sum(y-out)^2)
            loss_all += loss.numpy()  # 将每个step计算出的loss累加，为后续求loss平均值提供数据，这样计算的loss更准确
        # 计算loss对各个参数的梯度
        grads = tape.gradient(loss, [w1, b1])

        # 实现梯度更新 w1 = w1 - lr * w1_grad    b = b - lr * b_grad
        w1.assign_sub(lr * grads[0])  # 参数w1自更新
        b1.assign_sub(lr * grads[1])  # 参数b自更新

    # 每个epoch，打印loss信息
    print("Epoch {}, loss: {}".format(epoch, loss_all/4))
    train_loss_results.append(loss_all / 4)  # 将4个step的loss求平均记录在此变量中
    loss_all = 0  # loss_all归零，为记录下一个epoch的loss做准备

    # 测试部分
    # total_correct为预测对的样本个数, total_number为测试的总样本数，将这两个变量都初始化为0
    total_correct, total_number = 0, 0
    for x_test, y_test in test_db:
        # 使用更新后的参数进行预测
        y = tf.matmul(x_test, w1) + b1
        y = tf.nn.softmax(y)
        pred = tf.argmax(y, axis=1)  # 返回y中最大值的索引，即预测的分类
        # 将pred转换为y_test的数据类型
        pred = tf.cast(pred, dtype=y_test.dtype)
        # 若分类正确，则correct=1，否则为0，将bool型的结果转换为int型
        correct = tf.cast(tf.equal(pred, y_test), dtype=tf.int32)
        # 将每个batch的correct数加起来
        correct = tf.reduce_sum(correct)
        # 将所有batch中的correct数加起来
        total_correct += int(correct)
        # total_number为测试的总样本数，也就是x_test的行数，shape[0]返回变量的行数
        total_number += x_test.shape[0]
    # 总的准确率等于total_correct/total_number
    acc = total_correct / total_number
    test_acc.append(acc)
    print("Test_acc:", acc)
    print("--------------------------")

InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [32,3] vs. [32,452] [Op:Sub]

In [ ]:
# 绘制 loss 曲线
plt.title('Loss Function Curve')  # 图片标题
plt.xlabel('Epoch')  # x轴变量名称
plt.ylabel('Loss')  # y轴变量名称
plt.plot(train_loss_results, label="$Loss$")  # 逐点画出trian_loss_results值并连线，连线图标是Loss
plt.legend()  # 画出曲线图标
plt.show()  # 画出图像

In [ ]:
# 绘制 Accuracy 曲线
plt.title('Acc Curve')  # 图片标题
plt.xlabel('Epoch')  # x轴变量名称
plt.ylabel('Acc')  # y轴变量名称
plt.plot(test_acc, label="$Accuracy$")  # 逐点画出test_acc值并连线，连线图标是Accuracy
plt.legend()
plt.show()